In [1]:
import os,sys,warnings,re,math,gc,time
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["KERAS_BACKEND"] = "tensorflow"
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
from glob import glob
from sklearn.metrics import roc_auc_score,auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import lightgbm as lgb
tf.get_logger().setLevel("ERROR")
%xmode Minimal

Exception reporting mode: Minimal


In [2]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_thousands_separator(",")
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(6)

polars.config.Config

In [3]:
path_to_train = "/home/sohail/Downloads/credit_risk/train"
path_to_test = "/home/sohail/Downloads/credit_risk/test"
path_to_features = "feature_definitions.csv"

In [4]:
feat_df = pl.read_csv(path_to_features)
feat_df.head()

Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


In [5]:
all_train_files = glob(path_to_train+"/*.parquet")
all_test_files = glob(path_to_test+"/*.parquet")
train_files_df = pl.DataFrame({"index":range(len(all_train_files)),"path":all_train_files})
test_files_df = pl.DataFrame({"index":range(len(all_test_files)),"path":all_test_files})
train_files_df = (
    train_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
test_files_df = (
    test_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
display(train_files_df.head())
display(test_files_df.head())

index,path,filename
i64,str,str
31,"""/home/sohail/Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""/home/sohail/Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""/home/sohail/Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""
7,"""/home/sohail/Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""
11,"""/home/sohail/Downloads/credit_risk/train/train_credit_bureau_a_1_0.parquet""","""train_credit_bureau_a_1_0.parquet"""


index,path,filename
i64,str,str
31,"""/home/sohail/Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""/home/sohail/Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""/home/sohail/Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""/home/sohail/Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""/home/sohail/Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [6]:
train_base = (
    pl.read_parquet(path_to_train+"/train_base.parquet")
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("Date"),
        pl.col("target").cast(pl.UInt8)
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.week().alias("week"),
        pl.col("Date").dt.weekday().alias("weekday"),
        (pl.col("Date").dt.year() - 2019).alias("year")
    )
    .select(~cs.contains("target"),cs.contains("target"))
)
train_base

case_id_base,Date,month,week,weekday,year,target
u32,date,i8,i8,i8,i32,u8
0,2019-01-03,1,1,4,0,0
1,2019-01-03,1,1,4,0,0
2,2019-01-04,1,1,5,0,0
…,…,…,…,…,…,…
"2,703,452",2020-10-05,10,41,1,1,0
"2,703,453",2020-10-05,10,41,1,1,0
"2,703,454",2020-10-05,10,41,1,1,0


In [7]:
test_base = (
    pl.read_parquet(path_to_test+"/test_base.parquet")
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("Date"),
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.week().alias("week"),
        pl.col("Date").dt.weekday().alias("weekday"),
        (pl.col("Date").dt.year() - 2019).alias("year"),
        pl.lit(0).cast(pl.UInt8).alias("target")
    )
)
test_base

case_id_base,Date,month,week,weekday,year,target
u32,date,i8,i8,i8,i32,u8
"57,543",2020-10-06,10,41,2,1,0
"57,549",2020-10-06,10,41,2,1,0
"57,551",2020-10-06,10,41,2,1,0
…,…,…,…,…,…,…
"57,632",2020-10-06,10,41,2,1,0
"57,633",2020-10-06,10,41,2,1,0
"57,634",2020-10-06,10,41,2,1,0


In [8]:
total_base = (
    pl.concat(
        [
            train_base,
            test_base
        ],
        how="vertical"
    )
)

In [9]:
def reduce_dtypes(df:pl.DataFrame):
    return (
        df
        .with_columns(
            cs.by_name("case_id").cast(pl.UInt32),
            cs.contains("num_group").cast(pl.UInt16),
            cs.ends_with("D").cast(pl.Date),
            cs.ends_with("T","M").cast(pl.String),
            cs.ends_with("P","A").cast(pl.Float32),
            (cs.ends_with("L") & cs.numeric()).cast(pl.Float32),
        )
    )

def grouping(df):
    return (
        df
        .group_by("case_id")
        .agg(
            cs.numeric().max(),
            (~cs.numeric()).drop_nulls().mode().first()
        )
    )

def preprocess(filter_string:str,prefix_string:str):
    train_files_list = train_files_df.filter(pl.col("filename").str.contains(filter_string))["path"].to_list()
    test_files_list = test_files_df.filter(pl.col("filename").str.contains(filter_string))["path"].to_list()
    with pl.StringCache():
        train_df = (
            pl.concat(
                [
                    pl.read_parquet(_).pipe(reduce_dtypes).pipe(grouping) for _ in train_files_list
                ]
            )
        )
        test_df = (
            pl.concat(
                [
                    pl.read_parquet(_).select(train_df.columns).cast(train_df.schema).pipe(grouping) for _ in test_files_list
                ]
            )
        )
    return (
        pl.concat(
            [
                train_df,
                test_df
            ],
            how="vertical_relaxed"
        )
        .pipe(lambda df: df.rename({_:f"{prefix_string}_{_}" for _ in df.columns if not _ == 'case_id'}))
    )

def select_low_catcols(df:pl.DataFrame,thresh1=200,thresh2=2):
    col_names = []
    for col_name in df.select(cs.categorical()).columns:
        num_ = df.select(pl.col(col_name).value_counts()).shape[0]
        if (num_ > thresh1) | (num_ < thresh2):
            col_names.append(col_name)
    return df.select(~cs.by_name(col_names))
    

def select_impuatable(df:pl.DataFrame,thresh=0.95):
    cols =  (
        df
        .select(pl.all().is_null().mean())
        .transpose(include_header=True)
        .filter(pl.col("column_0") < thresh)
        ["column"].to_list()
    )
    return df.select(cols)

In [10]:
total_past_shallow = preprocess("applprev_1","past_shallow")
total_past_shallow

case_id,past_shallow_actualdpd_943P,past_shallow_annuity_853A,past_shallow_byoccupationinc_3656910L,past_shallow_childnum_21L,past_shallow_credacc_actualbalance_314A,past_shallow_credacc_credlmt_575A,past_shallow_credacc_maxhisbal_375A,past_shallow_credacc_minhisbal_90A,past_shallow_credacc_transactions_402L,past_shallow_credamount_590A,past_shallow_currdebt_94A,past_shallow_downpmt_134A,past_shallow_mainoccupationinc_437A,past_shallow_maxdpdtolerance_577P,past_shallow_num_group1,past_shallow_outstandingdebt_522A,past_shallow_pmtnum_8L,past_shallow_revolvingaccount_394A,past_shallow_tenor_203L,past_shallow_approvaldate_319D,past_shallow_cancelreason_3545846M,past_shallow_creationdate_885D,past_shallow_credacc_status_367L,past_shallow_credtype_587L,past_shallow_dateactivated_425D,past_shallow_district_544M,past_shallow_dtlastpmt_581D,past_shallow_dtlastpmtallstes_3545839D,past_shallow_education_1138M,past_shallow_employedfrom_700D,past_shallow_familystate_726L,past_shallow_firstnonzeroinstldate_307D,past_shallow_inittransactioncode_279L,past_shallow_isbidproduct_390L,past_shallow_isdebitcard_527L,past_shallow_postype_4733339M,past_shallow_profession_152M,past_shallow_rejectreason_755M,past_shallow_rejectreasonclient_4145042M,past_shallow_status_219L
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u16,f32,f32,f32,f32,date,str,date,str,str,date,str,date,date,str,date,str,date,str,bool,bool,str,str,str,str,str
"1,663,422",0.00,"3,068.80",null,1.00,null,0.00,null,null,null,"40,000.00",0.00,0.00,"14,000.00",163.00,2,0.00,24.00,null,24.00,2013-11-06,"""a55475b1""",2012-09-28,null,"""COL""",2012-10-18,"""P41_138_103""",2016-02-24,2016-02-24,"""P97_36_170""",2008-02-15,"""MARRIED""",2013-12-07,"""POS""",false,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""K"""
"114,977",0.00,"3,633.60",null,0.00,null,0.00,null,null,null,"40,000.00",0.00,0.00,"42,000.00",1.00,2,0.00,24.00,null,24.00,2017-09-10,"""a55475b1""",2017-09-10,null,"""CAL""",2017-09-12,"""P217_160_113""",2018-09-20,2018-09-20,"""P97_36_170""",2017-07-15,"""MARRIED""",2017-10-11,"""CASH""",false,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""K"""
"1,404,133",0.00,"8,137.60",null,1.00,null,0.00,null,null,null,"72,994.00",0.00,0.00,"130,000.00",0.00,0,0.00,11.00,null,11.00,2018-03-27,"""a55475b1""",2018-03-27,null,"""COL""",2018-04-03,"""P131_33_167""",2018-12-27,2018-12-27,"""P33_146_175""",2007-01-15,"""MARRIED""",2018-04-27,"""POS""",false,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""K"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"101,146",0.00,"7,800.80",1.00,0.00,null,0.00,null,null,null,"200,000.00","10,500.46",0.00,"30,000.00",0.00,3,"11,158.20",48.00,null,48.00,2022-06-24,"""a55475b1""",2016-10-06,null,"""CAL""",2022-09-03,"""a55475b1""",null,2022-08-23,"""a55475b1""",2015-07-15,"""SINGLE""",2015-11-23,"""CASH""",false,null,"""P46_145_78""","""a55475b1""","""a55475b1""","""a55475b1""","""D"""
"101,005",0.00,"5,847.00",null,null,null,0.00,null,null,null,"150,000.00",null,0.00,"50,000.00",null,1,null,60.00,null,60.00,null,"""P94_109_143""",2022-09-12,null,"""CAL""",null,"""P39_22_51""",null,null,"""a55475b1""",null,null,2022-10-15,"""CASH""",false,null,"""P46_145_78""","""a55475b1""","""a55475b1""","""P94_109_143""","""D"""
"100,984",0.00,0.00,null,null,"1,816.71",0.00,"4,000.00",0.00,3.00,0.00,0.00,0.00,null,0.00,0,0.00,null,"840,473,664.00",null,2022-09-12,"""a55475b1""",2022-09-12,"""AC""","""REL""",null,"""a55475b1""",null,2022-09-20,"""a55475b1""",null,null,null,"""NDF""",false,true,"""P67_102_161""","""a55475b1""","""a55475b1""","""a55475b1""","""N"""


In [11]:
total_past_depth = preprocess("applprev_2","past_depth")
total_past_depth

case_id,past_depth_num_group1,past_depth_num_group2,past_depth_cacccardblochreas_147M,past_depth_conts_type_509L,past_depth_credacc_cards_status_52L
u32,u16,u16,str,str,str
"789,127",2,0,"""a55475b1""","""PRIMARY_MOBILE""",null
"1,335,968",3,1,"""a55475b1""","""PRIMARY_MOBILE""",null
"1,367,185",1,0,"""a55475b1""","""PRIMARY_MOBILE""",null
…,…,…,…,…,…
"57,549",0,2,null,"""PHONE""",null
"57,551",1,1,null,"""PRIMARY_MOBILE""",null
"57,552",0,2,null,"""PRIMARY_MOBILE""",null


In [12]:
total_static_base = preprocess("static_0","static_base")
total_static_base

case_id,static_base_actualdpdtolerance_344P,static_base_amtinstpaidbefduel24m_4187115A,static_base_annuity_780A,static_base_annuitynextmonth_57A,static_base_applicationcnt_361L,static_base_applications30d_658L,static_base_applicationscnt_1086L,static_base_applicationscnt_464L,static_base_applicationscnt_629L,static_base_applicationscnt_867L,static_base_avgdbddpdlast24m_3658932P,static_base_avgdbddpdlast3m_4187120P,static_base_avgdbdtollast24m_4525197P,static_base_avgdpdtolclosure24_3658938P,static_base_avginstallast24m_3658937A,static_base_avglnamtstart24m_4525187A,static_base_avgmaxdpdlast9m_3716943P,static_base_avgoutstandbalancel6m_4187114A,static_base_avgpmtlast12m_4525200A,static_base_clientscnt12m_3712952L,static_base_clientscnt3m_3712950L,static_base_clientscnt6m_3712949L,static_base_clientscnt_100L,static_base_clientscnt_1022L,static_base_clientscnt_1071L,static_base_clientscnt_1130L,static_base_clientscnt_136L,static_base_clientscnt_157L,static_base_clientscnt_257L,static_base_clientscnt_304L,static_base_clientscnt_360L,static_base_clientscnt_493L,static_base_clientscnt_533L,static_base_clientscnt_887L,static_base_clientscnt_946L,static_base_cntincpaycont9m_3716944L,…,static_base_credtype_322L,static_base_datefirstoffer_1144D,static_base_datelastinstal40dpd_247D,static_base_datelastunpaid_3546854D,static_base_disbursementtype_67L,static_base_dtlastpmtallstes_4499206D,static_base_equalitydataagreement_891L,static_base_equalityempfrom_62L,static_base_firstclxcampaign_1125D,static_base_firstdatedue_489D,static_base_inittransactioncode_186L,static_base_isbidproduct_1095L,static_base_isbidproductrequest_292L,static_base_isdebitcard_729L,static_base_lastactivateddate_801D,static_base_lastapplicationdate_877D,static_base_lastapprcommoditycat_1041M,static_base_lastapprcommoditytypec_5251766M,static_base_lastapprdate_640D,static_base_lastcancelreason_561M,static_base_lastdelinqdate_224D,static_base_lastrejectcommoditycat_161M,static_base_lastrejectcommodtypec_5251769M,static_base_lastrejectdate_50D,static_base_lastrejectreason_759M,static_base_lastrejectreasonclient_4145040M,static_base_lastrepayingdate_696D,static_base_lastst_736L,static_base_maxdpdinstldate_3546855D,static_base_opencred_647L,static_base_paytype1st_925L,static_base_paytype_783L,static_base_payvacationpostpone_4187118D,static_base_previouscontdistrict_112M,static_base_twobodfilling_608L,static_base_typesuite_864L,static_base_validfrom_1069D
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,str,date,date,date,str,date,bool,bool,date,date,str,bool,bool,bool,date,date,str,str,date,str,date,str,str,date,str,str,date,str,date,bool,str,str,date,str,str,str,date
"678,092",null,null,"1,186.40",0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,…,"""REL""",null,null,null,"""SBA""",null,null,null,null,null,"""POS""",false,null,false,null,null,"""a55475b1""","""a55475b1""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""",null,"""a55475b1""","""a55475b1""",null,null,null,null,"""OTHER""","""OTHER""",null,"""a55475b1""","""FO""",null,null
"1,559,592",0.00,0.00,"1,236.60",0.00,0.00,0.00,0.00,0.00,0.00,0.00,826.00,null,826.00,826.00,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,…,"""COL""",2008-04-03,2017-05-30,2016-02-06,"""SBA""",2018-07-27,null,null,null,2012-06-08,"""POS""",false,null,null,2015-02-11,2015-07-20,"""P159_130_59""","""a55475b1""",2015-02-06,"""a55475b1""",2016-02-06,"""P159_130_59""","""a55475b1""",2015-07-20,"""a55475b1""","""a55475b1""",null,"""D""",2016-01-06,false,"""OTHER""","""OTHER""",null,"""P204_99_158""","""FO""",null,null
"1,410,314",0.00,"65,652.60","4,456.60",0.00,0.00,0.00,0.00,0.00,0.00,4.00,4.00,4.00,null,5.00,"6,565.80",null,7.00,"34,436.38",null,0.00,0.00,0.00,

In [13]:
total_static_external = (
    preprocess("static_cb","static_external")
    .with_columns(
        pl.col("static_external_riskassesment_302T").str.split("%").list.gather([0,1]).apply(lambda x: (int(x[0])+int(x[1].split("-")[1]))/200)
    )
)
total_static_external

case_id,static_external_contractssum_5085716L,static_external_days120_123L,static_external_days180_256L,static_external_days30_165L,static_external_days360_512L,static_external_days90_310L,static_external_firstquarter_103L,static_external_for3years_128L,static_external_for3years_504L,static_external_for3years_584L,static_external_formonth_118L,static_external_formonth_206L,static_external_formonth_535L,static_external_forquarter_1017L,static_external_forquarter_462L,static_external_forquarter_634L,static_external_fortoday_1092L,static_external_forweek_1077L,static_external_forweek_528L,static_external_forweek_601L,static_external_foryear_618L,static_external_foryear_818L,static_external_foryear_850L,static_external_fourthquarter_440L,static_external_numberofqueries_373L,static_external_pmtaverage_3A,static_external_pmtaverage_4527227A,static_external_pmtaverage_4955615A,static_external_pmtcount_4527229L,static_external_pmtcount_4955617L,static_external_pmtcount_693L,static_external_pmtscount_423L,static_external_pmtssum_45A,static_external_secondquarter_766L,static_external_thirdquarter_1082L,static_external_assignmentdate_238D,static_external_assignmentdate_4527235D,static_external_assignmentdate_4955616D,static_external_birthdate_574D,static_external_dateofbirth_337D,static_external_dateofbirth_342D,static_external_description_5085714M,static_external_education_1103M,static_external_education_88M,static_external_maritalst_385M,static_external_maritalst_893M,static_external_requesttype_4525192L,static_external_responsedate_1012D,static_external_responsedate_4527233D,static_external_responsedate_4917613D,static_external_riskassesment_302T,static_external_riskassesment_940T
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,date,date,date,date,date,date,str,str,str,str,str,str,date,date,date,f64,str
"734,449",null,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.00,0.00,null,null,null,null,null,null,6.00,"30,519.00",0.00,0.00,null,null,null,1977-01-01,1977-01-01,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,2019-07-20,null,null,null,null
"2,594,354",null,1.00,1.00,1.00,4.00,1.00,1.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.00,4.00,"17,465.20",null,null,null,null,6.00,null,null,0.00,4.00,2009-03-17,null,null,1951-03-01,1951-03-01,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,2019-07-19,null,null,null,null
"825,962",null,1.00,1.00,0.00,2.00,1.00,1.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.00,2.00,null,null,null,null,null,null,null,null,0.00,2.00,null,null,null,null,1988-05-01,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""DEDUCTION_6""",null,2019-10-23,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,543","22,130.26",9.00,9.00,9.00,10.00,9.00,4.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.00,10.00,null,null,null,null,null,null,null,null,1.00,6.00,null,null,null,null,1996-08-01,null,"""2fc785b2""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,2020-10-20,null,null
"57,549",null,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.00,0.00,null,null,null,null,null,null,null,null,2.00,0.00,null,null,null,null,1992-03-01,null,"""2fc785b2""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,2020-10-20,null,null
"57,634",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"6,917.00",null,14.00,null,null,null,null,null,null,null,2017-12-15,null,null,null,"""a55475b1""","""a55475b1""",

In [14]:
total_person_shallow = preprocess("person_1","person_shallow")
total_person_shallow

case_id,person_shallow_childnum_185L,person_shallow_mainoccupationinc_384A,person_shallow_num_group1,person_shallow_personindex_1023L,person_shallow_persontype_1072L,person_shallow_persontype_792L,person_shallow_birth_259D,person_shallow_birthdate_87D,person_shallow_contaddr_district_15M,person_shallow_contaddr_matchlist_1032L,person_shallow_contaddr_smempladdr_334L,person_shallow_contaddr_zipcode_807M,person_shallow_education_927M,person_shallow_empl_employedfrom_271D,person_shallow_empl_employedtotal_800L,person_shallow_empl_industry_691L,person_shallow_empladdr_district_926M,person_shallow_empladdr_zipcode_114M,person_shallow_familystate_447L,person_shallow_gender_992L,person_shallow_housetype_905L,person_shallow_housingtype_772L,person_shallow_incometype_1044T,person_shallow_isreference_387L,person_shallow_language1_981M,person_shallow_maritalst_703L,person_shallow_registaddr_district_1083M,person_shallow_registaddr_zipcode_184M,person_shallow_relationshiptoclient_415T,person_shallow_relationshiptoclient_642T,person_shallow_remitter_829L,person_shallow_role_1084L,person_shallow_role_993L,person_shallow_safeguarantyflag_411L,person_shallow_sex_738L,person_shallow_type_25L
u32,f32,f32,u16,f32,f32,f32,date,date,str,bool,bool,str,str,date,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,str,str,bool,str,str,bool,str,str
"670,893",null,"20,000.00",3,1.00,4.00,4.00,1990-03-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2014-01-15,"""MORE_FIVE""","""OTHER""","""a55475b1""","""a55475b1""","""MARRIED""",null,null,null,"""SALARIED_GOVT""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""SPOUSE""","""SPOUSE""",false,"""PE""",null,true,"""F""","""PRIMARY_MOBILE"""
"2,696,344",null,"40,000.00",0,0.00,1.00,1.00,1973-05-01,null,"""P121_27_170""",false,false,"""P39_16_16""","""a55475b1""",2016-08-20,null,null,"""a55475b1""","""a55475b1""",null,null,"""OWNED""",null,"""EMPLOYED""",null,"""P10_39_147""",null,"""P121_27_170""","""P39_16_16""",null,null,null,"""CL""",null,true,"""F""","""PRIMARY_MOBILE"""
"1,027,286",null,"72,000.00",1,1.00,4.00,4.00,1960-07-01,null,"""P153_41_170""",false,false,"""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""","""SINGLE""",null,null,null,"""SALARIED_GOVT""",null,"""a55475b1""",null,"""P153_41_170""","""P138_102_156""","""SPOUSE""","""SPOUSE""",false,"""PE""",null,false,"""M""","""PHONE"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,551",null,"24,000.00",2,1.00,5.00,5.00,1990-08-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2019-01-15,"""MORE_FIVE""","""OTHER""","""a55475b1""","""a55475b1""","""MARRIED""",null,null,null,"""EMPLOYED""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""FRIEND""","""FRIEND""",false,"""PE""",null,false,"""M""","""PHONE"""
"57,549",null,"15,000.00",3,1.00,5.00,5.00,1992-03-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2019-05-04,"""MORE_ONE""","""OTHER""","""a55475b1""","""a55475b1""","""SINGLE""",null,null,null,"""EMPLOYED""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""SIBLING""","""SIBLING""",false,"""EM""",null,false,"""M""","""PRIMARY_MOBILE"""
"57,543",null,"36,000.00",2,1.00,5.00,5.00,1996-08-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2018-02-15,"""MORE_ONE""","""OTHER""","""a55475b1""","""a55475b1""","""MARRIED""",null,null,null,"""EMPLOYED""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""OTHER""","""OTHER""",false,"""EM""",null,false,"""M""","""PHONE"""


In [15]:
total_person_depth = preprocess("person_2","person_depth")
total_person_depth

case_id,person_depth_num_group1,person_depth_num_group2,person_depth_addres_district_368M,person_depth_addres_role_871L,person_depth_addres_zip_823M,person_depth_conts_role_79M,person_depth_empls_economicalst_849M,person_depth_empls_employedfrom_796D,person_depth_empls_employer_name_740M,person_depth_relatedpersons_role_762T
u32,u16,u16,str,str,str,str,str,date,str,str
"845,073",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"958,719",1,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"2,545,988",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
…,…,…,…,…,…,…,…,…,…,…
"57,633",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"57,631",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"57,636",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null


In [16]:
total_other_shallow = preprocess("other_1","other_shallow")
total_other_shallow

case_id,other_shallow_amtdebitincoming_4809443A,other_shallow_amtdebitoutgoing_4809440A,other_shallow_amtdepositbalance_4809441A,other_shallow_amtdepositincoming_4809444A,other_shallow_amtdepositoutgoing_4809442A,other_shallow_num_group1
u32,f32,f32,f32,f32,f32,u16
"2,695,269","2,533.40","2,533.40",0.00,0.00,0.00,0
"253,004",96.40,0.00,0.00,0.00,0.00,0
"2,702,245","1,848.80","1,848.80",0.00,0.00,0.00,0
…,…,…,…,…,…,…
"57,694",0.00,0.00,"246,588.20",0.00,"2,294.80",0
"57,932",0.00,0.00,"23,212.00",0.00,178.20,0
"57,719",0.00,0.00,507.00,0.00,3.80,0


In [17]:
total_deposit_shallow = preprocess("deposit_1","deposit_shallow")
total_deposit_shallow

case_id,deposit_shallow_amount_416A,deposit_shallow_num_group1,deposit_shallow_contractenddate_991D,deposit_shallow_openingdate_313D
u32,f32,u16,date,date
"1,492,237","317,378.91",1,null,2016-06-27
"134,883",0.00,0,2017-06-17,2015-06-18
"2,683,086",505.06,1,2018-02-09,2014-02-28
…,…,…,…,…
"57,737",202.00,0,2018-05-16,2016-05-13
"58,038","78,590.74",1,null,2016-03-04
"57,968","2,025.38",0,null,2017-05-08


In [18]:
total_debitcard_shallow = preprocess("debitcard","card_shallow")
total_debitcard_shallow

case_id,card_shallow_last180dayaveragebalance_704A,card_shallow_last180dayturnover_1134A,card_shallow_last30dayturnover_651A,card_shallow_num_group1,card_shallow_openingdate_857D
u32,f32,f32,f32,u16,date
"1,596,436",0.00,79.88,0.00,2,2016-01-28
"1,835,274",null,null,null,0,2016-03-08
"205,954",null,null,null,1,2016-03-18
…,…,…,…,…,…
"57,925",null,null,null,0,2017-04-07
"57,932",null,null,null,0,2017-07-06
"58,038",null,null,null,1,2016-03-04


In [19]:
total_credit_internal_shallow = preprocess("bureau_a_1","int_shallow")
total_credit_internal_shallow

case_id,int_shallow_annualeffectiverate_199L,int_shallow_annualeffectiverate_63L,int_shallow_contractsum_5085717L,int_shallow_credlmt_230A,int_shallow_credlmt_935A,int_shallow_debtoutstand_525A,int_shallow_debtoverdue_47A,int_shallow_dpdmax_139P,int_shallow_dpdmax_757P,int_shallow_instlamount_768A,int_shallow_instlamount_852A,int_shallow_interestrate_508L,int_shallow_monthlyinstlamount_332A,int_shallow_monthlyinstlamount_674A,int_shallow_nominalrate_281L,int_shallow_nominalrate_498L,int_shallow_num_group1,int_shallow_numberofcontrsvalue_258L,int_shallow_numberofcontrsvalue_358L,int_shallow_numberofinstls_229L,int_shallow_numberofinstls_320L,int_shallow_numberofoutstandinstls_520L,int_shallow_numberofoutstandinstls_59L,int_shallow_numberofoverdueinstlmax_1039L,int_shallow_numberofoverdueinstlmax_1151L,int_shallow_numberofoverdueinstls_725L,int_shallow_numberofoverdueinstls_834L,int_shallow_outstandingamount_354A,int_shallow_outstandingamount_362A,int_shallow_overdueamount_31A,int_shallow_overdueamount_659A,int_shallow_overdueamountmax2_14A,int_shallow_overdueamountmax2_398A,int_shallow_overdueamountmax_155A,int_shallow_overdueamountmax_35A,int_shallow_periodicityofpmts_1102L,…,int_shallow_totalamount_6A,int_shallow_totalamount_996A,int_shallow_totaldebtoverduevalue_178A,int_shallow_totaldebtoverduevalue_718A,int_shallow_totaloutstanddebtvalue_39A,int_shallow_totaloutstanddebtvalue_668A,int_shallow_classificationofcontr_13M,int_shallow_classificationofcontr_400M,int_shallow_contractst_545M,int_shallow_contractst_964M,int_shallow_dateofcredend_289D,int_shallow_dateofcredend_353D,int_shallow_dateofcredstart_181D,int_shallow_dateofcredstart_739D,int_shallow_dateofrealrepmt_138D,int_shallow_description_351M,int_shallow_dpdmaxdatemonth_442T,int_shallow_dpdmaxdatemonth_89T,int_shallow_dpdmaxdateyear_596T,int_shallow_dpdmaxdateyear_896T,int_shallow_financialinstitution_382M,int_shallow_financialinstitution_591M,int_shallow_lastupdate_1112D,int_shallow_lastupdate_388D,int_shallow_numberofoverdueinstlmaxdat_148D,int_shallow_numberofoverdueinstlmaxdat_641D,int_shallow_overdueamountmax2date_1002D,int_shallow_overdueamountmax2date_1142D,int_shallow_overdueamountmaxdatemonth_284T,int_shallow_overdueamountmaxdatemonth_365T,int_shallow_overdueamountmaxdateyear_2T,int_shallow_overdueamountmaxdateyear_994T,int_shallow_purposeofcred_426M,int_shallow_purposeofcred_874M,int_shallow_refreshdate_3813885D,int_shallow_subjectrole_182M,int_shallow_subjectrole_93M
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,str,str,str,str,date,date,date,date,date,str,str,str,str,str,str,str,date,date,date,date,date,date,str,str,str,str,str,str,date,str,str
"1,350,152",null,0.12,null,null,"70,000.00","115,968.08",0.00,0.00,null,"9,818.60",null,null,"9,818.60",null,0.12,null,16,4.00,11.00,null,12.00,null,10.00,12.00,null,0.00,null,null,"28,336.04",null,0.00,"6,929.05",null,0.00,null,null,…,null,"33,998.00",0.00,0.00,"115,968.08",0.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2020-04-03,null,null,2019-01-14,null,"""a55475b1""",null,"""2.0""","""2018.0""",null,"""a55475b1""","""a55475b1""",2019-04-23,null,null,2015-04-14,null,2015-03-13,null,"""5.0""","""2018.0""",null,"""a55475b1""","""a55475b1""",2019-05-08,"""a55475b1""","""a55475b1"""
"2,558,305",null,null,null,null,"170,000.00","179,641.42",0.00,0.00,null,"6,980.60",null,null,"6,980.60",null,40.00,null,10,2.00,8.00,null,24.00,null,11.00,1.00,null,0.00,null,null,"39,168.69",null,0.00,"2,833.29",null,0.00,null,null,…,null,"69,504.00",0.00,0.00,"179,641.42",0.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2020-02-11,null,null,2018-02-12,null,"""a55475b1""",null,"""3.0""","""2017.0""",null,"""a55475b1""","""a55475b1""",2019-04-10,null,null,2016-12-24,null,2016-12-24,null,"""5.0""","""2018.0""",null,"""a55475b1""","""a55475b1""",2019-04-17,"""a55475b1""","""a5547

In [20]:
total_credit_internal_depth = preprocess("bureau_a_2","int_depth")
total_credit_internal_depth

case_id,int_depth_collater_valueofguarantee_1124L,int_depth_collater_valueofguarantee_876L,int_depth_num_group1,int_depth_num_group2,int_depth_pmts_dpd_1073P,int_depth_pmts_dpd_303P,int_depth_pmts_overdue_1140A,int_depth_pmts_overdue_1152A,int_depth_collater_typofvalofguarant_298M,int_depth_collater_typofvalofguarant_407M,int_depth_collaterals_typeofguarante_359M,int_depth_collaterals_typeofguarante_669M,int_depth_pmts_month_158T,int_depth_pmts_month_706T,int_depth_pmts_year_1139T,int_depth_pmts_year_507T,int_depth_subjectroles_name_541M,int_depth_subjectroles_name_838M
u32,f32,f32,u16,u16,f32,f32,f32,f32,str,str,str,str,str,str,str,str,str,str
"1,259,961",0.00,null,1,35,0.00,null,0.00,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""9.0""",null,"""2019.0""",null,"""a55475b1""","""a55475b1"""
"1,265,725",0.00,null,3,35,0.00,null,0.00,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""10.0""",null,"""2019.0""",null,"""a55475b1""","""a55475b1"""
"1,292,604",0.00,null,1,35,0.00,null,0.00,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""4.0""",null,"""2019.0""",null,"""a55475b1""","""a55475b1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"82,797",0.00,0.00,0,9,0.00,0.00,0.00,0.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""8.0""","""10.0""","""2020.0""","""2018.0""","""a55475b1""","""a55475b1"""
"85,784",0.00,null,0,9,0.00,null,0.00,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""10.0""",null,"""2020.0""",null,"""a55475b1""","""a55475b1"""
"91,257",0.00,0.00,0,9,0.00,0.00,0.00,0.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""3.0""","""4.0""","""2020.0""","""2017.0""","""a55475b1""","""a55475b1"""


In [21]:
total_credit_external_shallow = preprocess("bureau_b_1","ext_shallow")
total_credit_external_shallow

case_id,ext_shallow_amount_1115A,ext_shallow_credlmt_1052A,ext_shallow_credlmt_228A,ext_shallow_credlmt_3940954A,ext_shallow_credquantity_1099L,ext_shallow_credquantity_984L,ext_shallow_debtpastduevalue_732A,ext_shallow_debtvalue_227A,ext_shallow_dpd_550P,ext_shallow_dpd_733P,ext_shallow_dpdmax_851P,ext_shallow_installmentamount_644A,ext_shallow_installmentamount_833A,ext_shallow_instlamount_892A,ext_shallow_interesteffectiverate_369L,ext_shallow_interestrateyearly_538L,ext_shallow_maxdebtpduevalodued_3940955A,ext_shallow_num_group1,ext_shallow_numberofinstls_810L,ext_shallow_overdueamountmax_950A,ext_shallow_pmtdaysoverdue_1135P,ext_shallow_pmtnumpending_403L,ext_shallow_residualamount_1093A,ext_shallow_residualamount_127A,ext_shallow_residualamount_3940956A,ext_shallow_totalamount_503A,ext_shallow_totalamount_881A,ext_shallow_classificationofcontr_1114M,ext_shallow_contractdate_551D,ext_shallow_contractmaturitydate_151D,ext_shallow_contractst_516M,ext_shallow_contracttype_653M,ext_shallow_credor_3940957M,ext_shallow_dpdmaxdatemonth_804T,ext_shallow_dpdmaxdateyear_742T,ext_shallow_lastupdate_260D,ext_shallow_overdueamountmaxdatemonth_494T,ext_shallow_overdueamountmaxdateyear_432T,ext_shallow_periodicityofpmts_997L,ext_shallow_periodicityofpmts_997M,ext_shallow_pmtmethod_731M,ext_shallow_purposeofcred_722M,ext_shallow_subjectrole_326M,ext_shallow_subjectrole_43M
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,date,date,str,str,str,str,str,date,str,str,str,str,str,str,str,str
"836,028","9,516.00",0.00,"6,000.00",0.00,1.00,8.00,0.00,"2,379.00",0.00,0.00,0.00,0.00,"2,379.00","2,379.00",null,0.00,0.00,1,4.00,0.00,0.00,1.00,0.00,0.00,0.00,"9,516.00","124,444.40","""ea6782cc""",2011-05-22,2019-11-11,"""7241344e""","""1c9c5356""","""b619fa46""","""6.0""","""2011.0""",2019-10-26,"""8.0""","""2011.0""",null,"""a0b598e4""","""f6e26148""","""60c73645""","""ab3c25cf""","""ab3c25cf"""
"111,824","55,167.60",null,null,null,1.00,1.00,0.00,"35,647.43",0.00,0.00,"17,676.00",0.00,"35,647.43","4,216.00",null,43.00,0.60,0,18.00,0.60,0.00,10.00,null,null,null,"55,167.60","11,955.80","""ea6782cc""",2018-06-04,2019-12-05,"""7241344e""","""4257cbed""","""P0_31_66""","""2.0""","""2019.0""",2019-03-01,"""2.0""","""2019.0""",null,"""a0b598e4""","""f6e26148""","""60c73645""","""ab3c25cf""","""ab3c25cf"""
"1,391,167","20,338.40","8,040.00",0.00,"8,040.00",1.00,3.00,0.00,"11,863.40",0.00,0.00,0.00,0.00,"11,863.40","1,695.00",null,0.00,0.00,1,12.00,0.00,0.00,7.00,0.00,"5,359.80","5,359.80","20,338.40","237,079.28","""ea6782cc""",2018-12-19,2021-05-17,"""7241344e""","""4257cbed""","""b619fa46""","""1.0""","""2019.0""",2019-06-14,"""1.0""","""2017.0""",null,"""a55475b1""","""a55475b1""","""60c73645""","""ab3c25cf""","""ab3c25cf"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,679","60,000.00",0.00,null,0.00,1.00,2.00,0.00,"35,481.71",0.00,0.00,"17,310.00",0.00,"35,481.71","3,529.16",null,null,41.60,1,24.00,43.00,0.00,12.00,null,0.00,0.00,"60,000.00","86,800.00","""ea6782cc""",2019-06-26,2021-10-04,"""7241344e""","""4257cbed""","""50babcd4""","""9.0""","""2020.0""",2020-10-04,"""10.0""","""2020.0""",null,"""a0b598e4""","""dbcbe8f8""","""60c73645""","""ab3c25cf""","""a55475b1"""
"57,660",null,"133,279.00",null,"113,279.00",2.00,1.00,0.00,null,0.00,0.00,"32,618.00",0.00,"109,099.58",null,null,null,11.40,1,null,16.20,0.00,null,null,"109,099.58","90,845.98","133,279.00","16,800.00","""ea6782cc""",2019-12-08,2021-03-14,"""7241344e""","""1c9c5356""","""b619fa46""","""12.0""","""2019.0""",2020-10-20,"""8.0""","""2019.0""",null,"""a55475b1""","""a55475b1""","""60c73645""","""ab3c25cf""","""ab3c25cf"""
"57,767",null,0.00,"64,000.00",0.00,1.00,2.00,0.00,null,0.00,0.00,0.00,0.00,0.00,null,null,null,0.00,0,null,0.00,0.00,null,0.00,0.00,0.00,0.00,"64,000.00","""ea6782cc""",2013-10-14,2021-10-01,"""7241344e""","""1c9c5356""","""b619fa46""","""11.0""","""2013.0"

In [22]:
total_credit_external_depth = preprocess("bureau_b_2","ext_depth")
total_credit_external_depth

case_id,ext_depth_num_group1,ext_depth_num_group2,ext_depth_pmts_dpdvalue_108P,ext_depth_pmts_pmtsoverdue_635A,ext_depth_pmts_date_1107D
u32,u16,u16,f32,f32,date
"36,862",0,3,0.00,0.00,2019-09-15
"183,127",1,6,0.00,0.00,2019-11-15
"1,884,030",0,8,0.00,0.00,2020-07-15
…,…,…,…,…,…
"1,607,294",2,30,0.00,0.00,2019-07-15
"207,487",0,5,0.00,0.00,2020-01-15
"57,660",1,34,"8,814.00",2.20,2020-08-15


In [23]:
total_registry_a = preprocess("registry_a","reg_a")
total_registry_a

case_id,reg_a_amount_4527230A,reg_a_num_group1,reg_a_name_4527232M,reg_a_recorddate_4527225D
u32,f32,u16,str,date
"1,692,516","12,263.60",11,"""41493a12""",2019-12-31
"943,517",850.00,2,"""100201b1""",2020-02-07
"863,833","5,041.40",7,"""1e028df3""",2019-11-30
…,…,…,…,…
"1,817,264","2,235.60",10,"""9a0cb0f0""",2020-03-29
"57,679","3,640.00",4,"""ba006408""",2020-10-20
"57,689","3,214.80",6,"""aac4edf4""",2020-10-20


In [24]:
total_registry_b = preprocess("registry_b","reg_b")
total_registry_b

case_id,reg_b_amount_4917619A,reg_b_num_group1,reg_b_deductiondate_4917603D,reg_b_name_4917606M
u32,f32,u16,date,str
"1,872,898","20,131.80",5,2020-05-25,"""3570e91a"""
"1,885,852","17,086.20",4,2020-05-14,"""78403fe2"""
"1,887,552","59,032.80",3,2020-07-06,"""bd1743f9"""
…,…,…,…,…
"1,909,429","134,820.00",5,2020-06-01,"""38f38455"""
"57,543","31,060.20",4,2020-09-17,"""d2142f5c"""
"57,549","8,760.60",11,2020-05-06,"""bf93e092"""


In [25]:
total_registry_c = preprocess("registry_c","reg_c")
total_registry_c

case_id,reg_c_num_group1,reg_c_pmtamount_36A,reg_c_employername_160M,reg_c_processingdate_168D
u32,u16,f32,str,date
"1,369,802",5,"20,404.87","""61b1e5e0""",2019-02-26
"706,150",4,"6,003.60","""4a8b84d8""",2019-01-04
"1,332,842",5,"21,563.95","""4096c98c""",2019-02-19
…,…,…,…,…
"647,137",5,"1,193.75","""3c947003""",2019-01-09
"1,285,628",5,600.00,"""cf91d96f""",2018-09-07
"1,354,448",5,"7,615.41","""af8c129e""",2019-01-07


In [26]:
drop_columns = [
    "Date",
    # "static_base_commnoinclast6m_3546845L",
    # "static_base_deferredmnthsnum_166L",
    # "static_base_mastercontrelectronic_519L",
    # "static_base_mastercontrexist_109L",
    # "static_base_bankacctype_710L",
    # "static_base_isdebitcard_729L",
    # "static_base_paytype1st_925L",
    # "static_base_paytype_783L",
    # "static_base_typesuite_864L",
    # "person_shallow_contaddr_matchlist_1032L",
    # "person_shallow_remitter_829L",
    # "person_depth_empls_employer_name_740M",
    # "int_shallow_subjectrole_182M",
    # "person_shallow_relationshiptoclient_642T",
    # "int_shallow_refreshdate_3813885D",
    # "int_shallow_monthlyinstlamount_674A",
    # "static_base_maxlnamtstart6m_4525199A",
    # "static_base_dtlastpmtallstes_4499206D",
    # "static_base_cardtype_51L",
    # "static_base_applicationscnt_1086L",
    # "past_shallow_profession_152M",
    # "int_shallow_periodicityofpmts_1102L",
    # "past_shallow_annuity_853A",
    # "card_shallow_openingdate_857D",
    # "deposit_shallow_openingdate_313D",
    # "int_shallow_credlmt_230A",
    # "static_base_avgpmtlast12m_4525200A",
    # "int_shallow_monthlyinstlamount_332A",
    # "static_base_maxinstallast24m_3658928A",
    # "int_shallow_outstandingamount_354A",
    # "static_base_clientscnt_946L",
    # "int_shallow_credlmt_935A",
    # "int_shallow_dpdmaxdatemonth_442T",
    # "int_shallow_periodicityofpmts_837L",
    # "static_base_clientscnt_304L",
    # "int_shallow_totaloutstanddebtvalue_668A",
    # "int_shallow_residualamount_488A",
    # "person_shallow_contaddr_smempladdr_334L",
    # "person_depth_conts_role_79M",
    # "int_shallow_subjectrole_93M",
    # "int_shallow_contractsum_5085717L",
    # "past_depth_cacccardblochreas_147M",
    # "int_shallow_overdueamount_659A",
    # "int_shallow_debtoverdue_47A",
    # "int_shallow_totaldebtoverduevalue_178A",
    # "int_shallow_overdueamountmaxdatemonth_284T",
    # "person_shallow_empladdr_district_926M",
    # "person_depth_empls_economicalst_849M",
    # "static_base_maxpmtlast3m_4525190A",
    # "past_shallow_isbidproduct_390L",
    # "person_shallow_housetype_905L",
    # "person_shallow_empladdr_zipcode_114M",
    # "int_shallow_overdueamountmax_155A",
    # "int_shallow_overdueamountmax2_14A",
    # "int_shallow_financialinstitution_591M",
    # "static_base_totinstallast1m_4525188A",
    # "static_base_lastapprcommoditycat_1041M",
    # "static_base_applicationcnt_361L",
    # "person_shallow_safeguarantyflag_411L",
    # "person_depth_addres_zip_823M",
    # "person_depth_addres_district_368M",
    # "int_shallow_overdueamountmax_35A",
    # "static_base_clientscnt_493L",
    # "int_shallow_overdueamountmax2_398A",
    # "past_shallow_actualdpd_943P",
    # "past_depth_credacc_cards_status_52L",
    # "person_depth_num_group2",
    # "static_base_actualdpdtolerance_344P",
    ]

In [27]:
total_df = (
    total_base
    .join(
        total_past_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_past_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_static_base,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_static_external,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_person_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_person_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_other_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_deposit_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_debitcard_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_internal_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_external_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_external_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_a,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_b,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_c,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .with_columns(
        (pl.col(pl.Date) - pl.col("Date")).dt.total_days(),
        pl.col(pl.String).cast(pl.Categorical)
    )
    .drop(drop_columns)
    .pipe(select_impuatable)
    .pipe(select_low_catcols)
    .with_columns(
        pl.col(pl.INTEGER_DTYPES).fill_null(strategy="mean"),
        pl.col(pl.FLOAT_DTYPES).fill_null(strategy="mean"),
        pl.col(pl.Boolean).fill_null(value=pl.col(pl.Boolean).drop_nulls().mode().first()),
        pl.col(pl.Categorical).fill_null(value=pl.col(pl.Categorical).drop_nulls().mode().first())
    )
    .with_columns(
        pl.col(pl.Categorical).to_physical()
    )
    .with_columns(
        (pl.col(pl.INTEGER_DTYPES).exclude(["case_id_base","target"]) - pl.col(pl.INTEGER_DTYPES).exclude(["case_id_base","target"]).min())/(pl.col(pl.INTEGER_DTYPES).exclude(["case_id_base","target"]).max()-pl.col(pl.INTEGER_DTYPES).exclude(["case_id_base","target"]).min()),
        (pl.col(pl.FLOAT_DTYPES) - pl.col(pl.FLOAT_DTYPES).mean())/(pl.col(pl.FLOAT_DTYPES).std())
    )
    .select(
        ~cs.by_name("target"),
        cs.by_name("target")
    )
)
total_df

case_id_base,month,week,weekday,year,past_shallow_actualdpd_943P,past_shallow_annuity_853A,past_shallow_byoccupationinc_3656910L,past_shallow_childnum_21L,past_shallow_credacc_actualbalance_314A,past_shallow_credacc_credlmt_575A,past_shallow_credacc_maxhisbal_375A,past_shallow_credacc_minhisbal_90A,past_shallow_credacc_transactions_402L,past_shallow_credamount_590A,past_shallow_currdebt_94A,past_shallow_downpmt_134A,past_shallow_mainoccupationinc_437A,past_shallow_maxdpdtolerance_577P,past_shallow_num_group1,past_shallow_outstandingdebt_522A,past_shallow_pmtnum_8L,past_shallow_revolvingaccount_394A,past_shallow_tenor_203L,past_shallow_approvaldate_319D,past_shallow_cancelreason_3545846M,past_shallow_creationdate_885D,past_shallow_credacc_status_367L,past_shallow_credtype_587L,past_shallow_dateactivated_425D,past_shallow_dtlastpmt_581D,past_shallow_dtlastpmtallstes_3545839D,past_shallow_education_1138M,past_shallow_employedfrom_700D,past_shallow_familystate_726L,past_shallow_firstnonzeroinstldate_307D,past_shallow_inittransactioncode_279L,…,int_shallow_dateofcredend_289D,int_shallow_dateofcredend_353D,int_shallow_dateofcredstart_181D,int_shallow_dateofcredstart_739D,int_shallow_dateofrealrepmt_138D,int_shallow_description_351M,int_shallow_dpdmaxdatemonth_442T,int_shallow_dpdmaxdatemonth_89T,int_shallow_dpdmaxdateyear_596T,int_shallow_dpdmaxdateyear_896T,int_shallow_financialinstitution_382M,int_shallow_financialinstitution_591M,int_shallow_lastupdate_1112D,int_shallow_lastupdate_388D,int_shallow_numberofoverdueinstlmaxdat_148D,int_shallow_numberofoverdueinstlmaxdat_641D,int_shallow_overdueamountmax2date_1002D,int_shallow_overdueamountmax2date_1142D,int_shallow_overdueamountmaxdatemonth_284T,int_shallow_overdueamountmaxdatemonth_365T,int_shallow_overdueamountmaxdateyear_2T,int_shallow_overdueamountmaxdateyear_994T,int_shallow_purposeofcred_426M,int_shallow_purposeofcred_874M,int_shallow_refreshdate_3813885D,int_shallow_subjectrole_182M,int_shallow_subjectrole_93M,reg_a_amount_4527230A,reg_a_num_group1,reg_a_recorddate_4527225D,reg_b_amount_4917619A,reg_b_num_group1,reg_b_deductiondate_4917603D,reg_c_num_group1,reg_c_pmtamount_36A,reg_c_processingdate_168D,target
u32,f64,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f32,f32,f32,f32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f32,f64,f64,f32,f64,f64,f64,f32,f64,u8
0,0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.00,0.00,0.00,0.00,0.78,0.00,0.78,0.20,0.50,0.78,0.88,0.91,0.20,0.83,0.25,0.76,0.50,…,0.18,0.59,0.76,0.87,0.59,0.00,0.00,0.45,0.40,0.06,0.00,0.00,0.99,0.79,0.71,0.88,0.71,0.88,0.73,0.45,0.40,0.06,0.00,0.00,0.99,NaN,0.00,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.63,0
1,0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.00,0.00,0.00,0.00,0.78,0.00,0.78,0.20,0.50,0.78,0.88,0.91,0.20,0.83,0.25,0.76,0.50,…,0.18,0.59,0.76,0.87,0.59,0.00,0.00,0.45,0.40,0.06,0.00,0.00,0.99,0.79,0.71,0.88,0.71,0.88,0.73,0.45,0.40,0.06,0.00,0.00,0.99,NaN,0.00,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.63,0
2,0.00,0.00,0.67,0.00,-0.01,-1.08,0.00,-1.12,0.00,-0.45,0.00,0.00,0.00,-1.06,0.00,-0.35,-1.58,0.00,0.05,0.00,0.08,0.00,0.08,0.78,0.00,0.61,0.20,0.00,0.78,0.88,0.91,0.00,0.85,0.00,0.61,0.00,…,0.18,0.59,0.76,0.87,0.59,0.00,0.00,0.45,0.40,0.06,0.00,0.00,0.99,0.79,0.71,0.88,0.71,0.88,0.73,0.45,0.40,0.06,0.00,0.00,0.99,NaN,0.00,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.63,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,632",0.82,0.78,0.17,1.00,-0.01,0.19,0.00,0.00,0.00,-0.45,0.00,0.00,0.00,0.54,0.00,-0.35,0.99,0.00,0.00,0.00,0.08,0.00,0.08,0.78,0.12,1.00,0.20,0.00,0.78,0.88,0.91,0.20,0.83,0.25,1.00,0.00,…,0.18,0.59,0.76,0.87,0.59,0.00,0.00,0.45,0.40,0.06,0.00,0.0

In [28]:
total_df.write_parquet("total_df.parquet")

In [28]:
train_total = total_df.filter(pl.col("case_id_base").is_in(test_base.select("case_id_base")).not_()).drop("case_id_base")
train_total.head()

month,week,weekday,year,past_shallow_actualdpd_943P,past_shallow_annuity_853A,past_shallow_byoccupationinc_3656910L,past_shallow_childnum_21L,past_shallow_credacc_actualbalance_314A,past_shallow_credacc_credlmt_575A,past_shallow_credacc_maxhisbal_375A,past_shallow_credacc_minhisbal_90A,past_shallow_credacc_transactions_402L,past_shallow_credamount_590A,past_shallow_currdebt_94A,past_shallow_downpmt_134A,past_shallow_mainoccupationinc_437A,past_shallow_maxdpdtolerance_577P,past_shallow_num_group1,past_shallow_outstandingdebt_522A,past_shallow_pmtnum_8L,past_shallow_revolvingaccount_394A,past_shallow_tenor_203L,past_shallow_approvaldate_319D,past_shallow_cancelreason_3545846M,past_shallow_creationdate_885D,past_shallow_credacc_status_367L,past_shallow_credtype_587L,past_shallow_dateactivated_425D,past_shallow_dtlastpmt_581D,past_shallow_dtlastpmtallstes_3545839D,past_shallow_education_1138M,past_shallow_employedfrom_700D,past_shallow_familystate_726L,past_shallow_firstnonzeroinstldate_307D,past_shallow_inittransactioncode_279L,past_shallow_isbidproduct_390L,…,int_shallow_dateofcredend_289D,int_shallow_dateofcredend_353D,int_shallow_dateofcredstart_181D,int_shallow_dateofcredstart_739D,int_shallow_dateofrealrepmt_138D,int_shallow_description_351M,int_shallow_dpdmaxdatemonth_442T,int_shallow_dpdmaxdatemonth_89T,int_shallow_dpdmaxdateyear_596T,int_shallow_dpdmaxdateyear_896T,int_shallow_financialinstitution_382M,int_shallow_financialinstitution_591M,int_shallow_lastupdate_1112D,int_shallow_lastupdate_388D,int_shallow_numberofoverdueinstlmaxdat_148D,int_shallow_numberofoverdueinstlmaxdat_641D,int_shallow_overdueamountmax2date_1002D,int_shallow_overdueamountmax2date_1142D,int_shallow_overdueamountmaxdatemonth_284T,int_shallow_overdueamountmaxdatemonth_365T,int_shallow_overdueamountmaxdateyear_2T,int_shallow_overdueamountmaxdateyear_994T,int_shallow_purposeofcred_426M,int_shallow_purposeofcred_874M,int_shallow_refreshdate_3813885D,int_shallow_subjectrole_182M,int_shallow_subjectrole_93M,reg_a_amount_4527230A,reg_a_num_group1,reg_a_recorddate_4527225D,reg_b_amount_4917619A,reg_b_num_group1,reg_b_deductiondate_4917603D,reg_c_num_group1,reg_c_pmtamount_36A,reg_c_processingdate_168D,target
f64,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f32,f32,f32,f32,f64,u32,f64,u32,u32,f64,f64,f64,u32,f64,u32,f64,u32,bool,…,f64,f64,f64,f64,f64,u32,u32,u32,u32,u32,u32,u32,f64,f64,f64,f64,f64,f64,u32,u32,u32,u32,u32,u32,f64,u32,u32,f32,f64,f64,f32,f64,f64,f64,f32,f64,u8
0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.00,0.00,0.00,0.00,0.78,0,0.78,1,1,0.77,0.88,0.91,1,0.83,1,0.76,1,false,…,0.40,0.59,0.76,0.87,0.59,0,0,2,2,1,0,0,1.00,0.79,0.70,0.88,0.70,0.88,8,2,2,1,0,0,0.99,0,0,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.62,0
0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.00,0.00,0.00,0.00,0.78,0,0.78,1,1,0.77,0.88,0.91,1,0.83,1,0.76,1,false,…,0.40,0.59,0.76,0.87,0.59,0,0,2,2,1,0,0,1.00,0.79,0.70,0.88,0.70,0.88,8,2,2,1,0,0,0.99,0,0,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.62,0
0.00,0.00,0.67,0.00,-0.01,-1.08,0.00,-1.12,0.00,-0.45,0.00,0.00,0.00,-1.06,0.00,-0.35,-1.58,0.00,0.05,0.00,0.08,0.00,0.08,0.78,0,0.61,1,0,0.77,0.88,0.91,0,0.85,0,0.61,0,false,…,0.40,0.59,0.76,0.87,0.59,0,0,2,2,1,0,0,1.00,0.79,0.70,0.88,0.70,0.88,8,2,2,1,0,0,0.99,0,0,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.62,0
0.00,0.00,0.50,0.00,-0.01,0.13,0.00,0.00,0.00,-0.45,0.00,0.00,0.00,-0.22,0.00,-0.35,-1.50,0.00,0.00,0.00,-0.97,0.00,-0.97,0.78,1,1.00,1,0,0.77,0.88,0.91,0,0.99,1,1.00,0,false,…,0.40,0.59,0.76,0.87,0.59,0,0,2,2,1,0,0,1.00,0.79,0.70,0.88,0.70,0.88,8,2,2,1,0,0,0.99,0,0,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.62,0
0.00,0.00,0.67,0.00,-0.01,-0.84,0.00,0.00,0.00,-0.45,0.00,0.00,0.00,-0.60,0.00,-0.35,-1.36,0.00,0.00,0.00,0.08,0.00,0.08,0.78,2,1.00,1,0,0.77,0.88,0.91,1,0.83,1,1.00,0,false,…,0.40,0.59,0.76,0.87,0.59,0,0,2,2,1,0,0,1.00,0.79,0.70,0.88,0.70,0.8

In [29]:
submission_df = total_df.filter(pl.col("case_id_base").is_in(test_base.select("case_id_base"))).drop(["target"])
submission_df.head()

case_id_base,month,week,weekday,year,past_shallow_actualdpd_943P,past_shallow_annuity_853A,past_shallow_byoccupationinc_3656910L,past_shallow_childnum_21L,past_shallow_credacc_actualbalance_314A,past_shallow_credacc_credlmt_575A,past_shallow_credacc_maxhisbal_375A,past_shallow_credacc_minhisbal_90A,past_shallow_credacc_transactions_402L,past_shallow_credamount_590A,past_shallow_currdebt_94A,past_shallow_downpmt_134A,past_shallow_mainoccupationinc_437A,past_shallow_maxdpdtolerance_577P,past_shallow_num_group1,past_shallow_outstandingdebt_522A,past_shallow_pmtnum_8L,past_shallow_revolvingaccount_394A,past_shallow_tenor_203L,past_shallow_approvaldate_319D,past_shallow_cancelreason_3545846M,past_shallow_creationdate_885D,past_shallow_credacc_status_367L,past_shallow_credtype_587L,past_shallow_dateactivated_425D,past_shallow_dtlastpmt_581D,past_shallow_dtlastpmtallstes_3545839D,past_shallow_education_1138M,past_shallow_employedfrom_700D,past_shallow_familystate_726L,past_shallow_firstnonzeroinstldate_307D,past_shallow_inittransactioncode_279L,…,int_shallow_contractst_964M,int_shallow_dateofcredend_289D,int_shallow_dateofcredend_353D,int_shallow_dateofcredstart_181D,int_shallow_dateofcredstart_739D,int_shallow_dateofrealrepmt_138D,int_shallow_description_351M,int_shallow_dpdmaxdatemonth_442T,int_shallow_dpdmaxdatemonth_89T,int_shallow_dpdmaxdateyear_596T,int_shallow_dpdmaxdateyear_896T,int_shallow_financialinstitution_382M,int_shallow_financialinstitution_591M,int_shallow_lastupdate_1112D,int_shallow_lastupdate_388D,int_shallow_numberofoverdueinstlmaxdat_148D,int_shallow_numberofoverdueinstlmaxdat_641D,int_shallow_overdueamountmax2date_1002D,int_shallow_overdueamountmax2date_1142D,int_shallow_overdueamountmaxdatemonth_284T,int_shallow_overdueamountmaxdatemonth_365T,int_shallow_overdueamountmaxdateyear_2T,int_shallow_overdueamountmaxdateyear_994T,int_shallow_purposeofcred_426M,int_shallow_purposeofcred_874M,int_shallow_refreshdate_3813885D,int_shallow_subjectrole_182M,int_shallow_subjectrole_93M,reg_a_amount_4527230A,reg_a_num_group1,reg_a_recorddate_4527225D,reg_b_amount_4917619A,reg_b_num_group1,reg_b_deductiondate_4917603D,reg_c_num_group1,reg_c_pmtamount_36A,reg_c_processingdate_168D
u32,f64,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f32,f32,f32,f32,f64,u32,f64,u32,u32,f64,f64,f64,u32,f64,u32,f64,u32,…,u32,f64,f64,f64,f64,f64,u32,u32,u32,u32,u32,u32,u32,f64,f64,f64,f64,f64,f64,u32,u32,u32,u32,u32,u32,f64,u32,u32,f32,f64,f64,f32,f64,f64,f64,f32,f64
"57,543",0.82,0.78,0.17,1.00,-0.01,-1.12,0.00,0.00,0.00,-0.45,0.00,0.00,0.00,-0.64,0.00,-0.35,-0.97,0.00,0.00,0.00,0.70,0.00,0.70,0.78,1,0.85,1,1,0.77,0.88,0.91,1,0.83,1,0.84,1,…,0,0.39,0.59,0.76,0.82,0.59,0,0,2,3,1,0,0,1.00,0.79,0.70,1.00,0.70,0.95,8,5,0,1,0,0,0.67,0,0,0.00,0.06,0.91,-0.39,0.04,0.84,0.04,0.00,0.62
"57,549",0.82,0.78,0.17,1.00,-0.01,-0.98,0.00,0.00,0.00,-0.45,0.00,0.00,0.00,-0.98,0.00,-0.35,-1.47,0.00,0.00,0.00,-0.97,0.00,-0.97,0.78,1,0.90,1,0,0.77,0.88,0.91,0,0.94,0,0.90,0,…,1,0.40,0.61,0.64,0.87,0.60,0,7,2,2,16,5,0,1.00,0.98,0.98,0.88,0.92,0.88,2,2,2,14,0,3,0.99,0,1,0.00,0.06,0.91,-2.14,0.11,0.76,0.04,0.00,0.62
"57,551",0.82,0.78,0.17,1.00,-0.01,-0.07,0.00,0.00,0.00,-0.45,0.00,0.00,0.00,-0.22,0.00,-0.35,-0.41,0.00,0.05,0.00,-0.44,0.00,-0.44,0.78,1,0.94,1,0,0.77,0.88,0.91,0,0.95,1,0.94,0,…,0,0.40,0.59,0.76,0.87,0.59,0,0,2,2,1,0,0,1.00,0.79,0.70,0.88,0.70,0.88,8,2,2,1,0,0,0.99,0,0,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.62
"57,552",0.82,0.78,0.33,1.00,-0.01,-0.23,0.00,-0.28,0.00,-0.45,0.00,0.00,0.00,-0.41,0.00,-0.35,-1.36,0.00,0.00,0.00,-0.44,0.00,-0.44,0.78,1,0.75,1,0,0.77,0.88,0.91,0,0.83,1,0.75,0,…,0,0.40,0.59,0.76,0.87,0.59,0,0,2,2,1,0,0,1.00,0.79,0.70,0.88,0.70,0.88,8,2,2,1,0,0,0.99,0,0,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.62
"57,569",0.82,0.78,0.17,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.00,0.00,0.00,0.00,0.78,0,0.78,1,1,0.77,0.88,0.91,1,0.83,1,0.76,1,…,0,0.40,0.59,0.76,0.87,0.59,0,0,2,2

In [30]:
train_df,valid_df = train_test_split(
    train_total,
    stratify=train_total.select("target"),
    test_size=0.1,
    random_state=420,
    shuffle=True
    )

In [31]:
print(train_df.shape)
print(valid_df.shape)

(1373993, 351)
(152666, 351)


In [32]:
params = {
    "objective":"binary",
    "metric":"auc",
    "max_depth": 32,
    "learning_rate": 0.05,
    "n_estimators": 3000,
    "colsample_bynode": 0.9,
    "colsample_bytree": 0.9,
    "verbose": -1,
    "random_state": 42,
    "reg_alpha": 0.2,
    "reg_lambda": 10,
    "extra_trees": True,
    "num_leaves": 64,
    "device": "gpu",
    "verbose": -1
}

In [33]:
X_train,y_train = train_df.select((~(cs.by_name("target"))).cast(pl.Float32)),train_df.select(pl.col("target").cast(pl.Float32))
X_valid,y_valid = valid_df.select((~(cs.by_name("target"))).cast(pl.Float32)),valid_df.select(pl.col("target").cast(pl.Float32))

In [34]:
model = lgb.LGBMClassifier(**params)
model.fit(X_train,y_train,eval_set=[(X_valid,y_valid)],callbacks=[lgb.log_evaluation(100)])

[100]	valid_0's auc: 0.838438
[200]	valid_0's auc: 0.848866
[300]	valid_0's auc: 0.85368
[400]	valid_0's auc: 0.856222
[500]	valid_0's auc: 0.857579
[600]	valid_0's auc: 0.858419
[700]	valid_0's auc: 0.85884
[800]	valid_0's auc: 0.859397
[900]	valid_0's auc: 0.859706
[1000]	valid_0's auc: 0.860029
[1100]	valid_0's auc: 0.860197
[1200]	valid_0's auc: 0.860348
[1300]	valid_0's auc: 0.860624
[1400]	valid_0's auc: 0.860788
[1500]	valid_0's auc: 0.860881
[1600]	valid_0's auc: 0.861002
[1700]	valid_0's auc: 0.861038
[1800]	valid_0's auc: 0.86113
[1900]	valid_0's auc: 0.861207
[2000]	valid_0's auc: 0.861237
[2100]	valid_0's auc: 0.861228
[2200]	valid_0's auc: 0.861313
[2300]	valid_0's auc: 0.861311
[2400]	valid_0's auc: 0.861336
[2500]	valid_0's auc: 0.861278
[2600]	valid_0's auc: 0.861144
[2700]	valid_0's auc: 0.861124
[2800]	valid_0's auc: 0.861172
[2900]	valid_0's auc: 0.861084
[3000]	valid_0's auc: 0.861057


LGBMClassifier(colsample_bynode=0.8, colsample_bytree=0.8, device='gpu',
               extra_trees=True, learning_rate=0.05, max_depth=32, metric='auc',
               n_estimators=3000, num_leaves=64, objective='binary',
               random_state=42, reg_alpha=0.1, reg_lambda=10, verbose=-1)

In [35]:
train_df.select(pl.col("int_shallow_financialinstitution_591M").value_counts()).unnest(cs.all()).sort(by="count")

int_shallow_financialinstitution_591M,count
u32,u32
25,1
36,1
34,1
…,…
6,86
4,156
0,"1,373,406"


In [36]:
train_df.select(pl.col("target").value_counts()).unnest(cs.all())

target,count
u8,u32
0,"1,330,798"
1,"43,195"


In [37]:
for num,col_name in sorted(zip(model.feature_importances_,X_train.columns)):
    print(num,col_name)

0 int_shallow_subjectrole_182M
0 int_shallow_subjectrole_93M
0 past_depth_cacccardblochreas_147M
0 person_depth_empls_employer_name_740M
0 person_shallow_contaddr_matchlist_1032L
0 person_shallow_remitter_829L
0 static_base_applicationcnt_361L
0 static_base_bankacctype_710L
0 static_base_commnoinclast6m_3546845L
0 static_base_deferredmnthsnum_166L
0 static_base_isdebitcard_729L
0 static_base_mastercontrelectronic_519L
0 static_base_mastercontrexist_109L
0 static_base_paytype1st_925L
0 static_base_paytype_783L
0 static_base_typesuite_864L
2 static_external_assignmentdate_4527235D
5 deposit_shallow_num_group1
5 int_shallow_purposeofcred_426M
8 static_base_cardtype_51L
10 person_depth_empls_economicalst_849M
11 int_shallow_numberofoverdueinstls_834L
15 static_base_clientscnt3m_3712950L
17 person_shallow_housetype_905L
17 reg_a_recorddate_4527225D
23 static_base_lastrejectcommodtypec_5251769M
25 static_external_responsedate_4527233D
28 int_shallow_contractst_964M
29 int_shallow_numberofout

In [38]:
sub_df = pd.DataFrame({
    "case_id": submission_df['case_id_base'].to_list(),
    "score": model.predict_proba(submission_df.drop("case_id_base"))[:,1]
}).set_index("case_id")
sub_df.to_csv("./submission.csv")